# Dataset.map()

This notebook shows a workflow for using `Dataset.map`. This method is useful for creating a new column with a custom map function to generate the output.


In [1]:
import lilac as ll

ll.set_project_dir('./data')

try:
  glue = ll.get_dataset('local', 'glue_ax_map')
except:
  glue = ll.create_dataset(
    ll.DatasetConfig(
      namespace='local',
      name='glue_ax_map',
      source=ll.HuggingFaceSource(
        dataset_name='glue',
        config_name='ax',
      )))

#ll.start_server()


/Users/nikhil/Code/lilac/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Executing:
SELECT COUNT() as count FROM t
Query took 0.000s.


# Upper case 'premise'

The following map will upper case the 'premise' field from the dataset.

The output of the map is returned as a generator.


In [2]:
# Upper case 'premise' and print the first result
# This call does not save the output to a column.
res = glue.map(lambda item: item['premise'].upper())
print(next(iter(res)))

# Write the output to a column 'premise_upper'.
glue.map(lambda item: item['premise'].upper(), output_path='premise_upper')
glue.select_rows(['premise', 'premise_upper'], limit=1)


got output path None


Computing map over ['*', '__rowid__']: 100%|██████████| 1104/1104 [00:00<00:00, 72384.54it/s]

OP= None
THE CAT SAT ON THE MAT.


ValueError: Unable to select path ('premise_upper',). Path part "premise_upper" not found in the dataset.